In [4]:
import pandas as pd
import numpy as np
import random
!pip install statsmodels
!pip install scikit-learn==0.24
!pip install keras==2.6
#!pip3 install --force-reinstall tensorflow
#!pip install -q tensorflow-lattice pydot
#!pip install tensorflow-lattice-gpu
from statsmodels.tsa.stattools import pacf
from matplotlib.pyplot import figure
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_lattice as tfl
import matplotlib.ticker as ticker

     |████████████████████████████████| 1.3 MB 2.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you'll have keras 2.6.0 which is incompatible.


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

# alternative:
# scipy.stats.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

In [7]:
# import data
df = pd.read_csv("CACases.csv", index_col = False)
df.reset_index(drop=True, inplace=True)
df.head()

,date,Los Angeles,San Diego,San Francisco,Santa Barbara,Fresno,Sacramento,Ventura,Riverside
0,2020-02-01,13.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1,2020-02-02,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2020-02-03,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020-02-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# counties we are interested in
counties = ["Los Angeles", "San Diego", "San Francisco", "Santa Barbara",\
            "Fresno", "Sacramento", "Ventura", "Riverside"]

# Data Smoothing
- Remove day of week effect by replace $y_t$ with an average of 7 previous days(inclusive)
    - Each value is an average of data from all 7 different weekdays, thud day of week effect removed

In [9]:
# create a time series of cases, each value is an average of 7 previous days(include the current date)
# for each county
df1 = df.copy()  # deep copy
df1 = df1.iloc[6:]
for county in counties:
    # holder
    temp = []
    # take average, starting from 7th day
    for i in range(6,len(df)):
        # remove day of week effect
        ave = np.mean(df[county].iloc[i-6:i+1])  # 6 previous days and the exact date, 7 days' average
        temp.append(ave)
    df1[county] = temp
df1.set_index("date", inplace=True)

In [10]:
# input: original dataset, county we want
# output: all data, sorted by date, of that county
def extract(dataset, county):
    # select data only from the input county
    temp = dataset.to_dict()[county]
    temp = pd.Series(temp)
    # reset index
    # temp.reset_index(drop=True, inplace=True)
    return temp

# Data Preprocessing
## Differencing
- Differencing of time series in discrete time
- transformation of series to a new time series wehre values are the difference between consecutive values of previous series
- help stablizing the mean of time series by removing trend

In [11]:
# create a differenced series, output as Series
# interval: order of differencing, default = 1
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        # no need to difference
        if (interval == 0):
            value = dataset[i]
        else:
            value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)
# invert differenced value
# input: single original, single prediction, interval(default as 1)
def invert_difference(history, yhat, interval=1):
    if (interval == 0):
        return yhat
    else:
        return yhat + history

## Scaling
- Scale the data to [-1,1]

In [12]:
# scale train and test data to [-1, 1]
# input: train(np array, train_size*1), test(np array, test_size*1)
def scale(train, test):
    # find scalers
    aveTrain = np.mean(train)
    maxTrain = max(train)
    minTrain = min(train)
    scaler = [aveTrain, maxTrain, minTrain]
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = (train - aveTrain)/(maxTrain-minTrain)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = (test - aveTrain)/(maxTrain-minTrain)
    return scaler, train_scaled, test_scaled
# inverse scaling for a forecasted value
# input: scaler, single prediction yhat
def invert_scale(scaler, yhat):
    inverted = yhat*(scaler[1]-scaler[2]) + scaler[0]
    return inverted[0][0]  # output: a number

## Reshaping

In [13]:
# convert to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = [], []
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg  # output: a data frame
# prepare data for AR
def prepare_AR(series, diff, train_size, test_size, n_lag, n_seq):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), 1)
    # differencing
    diff_series = difference(raw_values, diff)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
    # split into train & test
    train_diff, test_diff = diff_values[:(train_size-diff)], diff_values[(train_size-diff):]
    # rescaling
    scaler, train_scaled, test_scaled = scale(train_diff, test_diff)
    # adjust data type for test_scaled
    # test_scaled = [row[0][0] for row in test_scaled]
    # convert to supervised
    train = [row[0][0] for row in train_scaled]
    test = series_to_supervised(test_scaled, n_lag, n_seq).values
    return scaler, train, test  # scaler: list, train: list, test: np array
# prepare data for neural networks
def prepare_nn(series, diff, train_size, test_size, n_lag, n_seq):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), 1)
    # differencing
    diff_series = difference(raw_values, diff)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
    # split into train & test
    train_diff, test_diff = diff_values[:(train_size-diff)], diff_values[(train_size-diff):]
    # rescaling
    scaler, train_scaled, test_scaled = scale(train_diff, test_diff)
    # adjust data type for train_scaled, test_scaled
    train_scaled = [row[0][0] for row in train_scaled]
    test_scaled = [row[0][0] for row in test_scaled]
    # convert to supervised
    train = series_to_supervised(train_scaled, n_lag, n_seq).values
    test = series_to_supervised(test_scaled, n_lag, n_seq).values
    return scaler, train, test  # scaler: list of np array, train: np array, test: np array

# Model Training

In [14]:
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = keras.Sequential()
    model.add(layers.LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(layers.Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [15]:
# fit an 2 layer LSTM network to training data
def fit_double_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = keras.Sequential()
    # two layers of lstm
    model.add(layers.LSTM(n_neurons, return_sequences=True, 
                          batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(layers.LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[2], X.shape[2]), stateful=True))
    model.add(layers.Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [16]:
class alpha_layer(keras.layers.Layer):
    def __init__(self, input_dim=1, units=1):
        super(alpha_layer, self).__init__()
        # check with initializer.get_config()
        initializer = tf.keras.initializers.RandomUniform(minval = 0, maxval = 1)
        self.alpha = self.add_weight(shape=(input_dim, units), initializer=initializer, 
                                     constraint=lambda x: tf.clip_by_value(x, 0, 1), trainable=True)

    def call(self, input1, input2):
        return tf.matmul(input1, self.alpha) + tf.matmul(input2, (1-self.alpha))

In [17]:
def fit_comb(train, n_lag, n_seq, n_batch, n_epoch, n_neurons):
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    Z = X.reshape(X.shape[0], 1, X.shape[1])
    # AR layer
    input1 = tf.keras.layers.Input(shape=(X.shape[1],))
    AR = tfl.layers.Linear(num_input_dims=X.shape[1], units=n_seq)(input1)
    
    # lstm layer
    input2 = tf.keras.layers.Input(shape=(Z.shape[1],Z.shape[2]))
    lstm = layers.LSTM(n_seq)(input2)
    # fully connected lstm
    lstm_connected = tf.keras.layers.Dense(n_seq)(lstm)
    
    # alpha layer
    layer = alpha_layer()
    #added = layer(AR, lstm)
    added = layer(AR, lstm_connected)
    model = tf.keras.models.Model(inputs=[input1,input2], outputs=added)
    # compile before training or testing
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(n_epoch):
        model.fit([X, Z], y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

# Make Forecasts

In [18]:
# evaluate the model
def make_AR(model, raw_value, test, test_size, n_lag, n_seq, diff):
    forecasts = []
    prev = raw_value[-(test_size+diff):(len(raw_value)-diff)]  # Y_(t-diff)  # (train_size+n_lag-1):-1
    for i in range(test_size):
        X = test[i, 0:n_lag]  # X_test, will not use y_test
        # make forecast
        forecast = model.params[0]  # initlialize with coefficient
        for j in range(n_lag):
            forecast += model.params[j+1] * X[j]  # forecast = a0 + a_i * X_i for i = 1,...,7
        # invert scaling
        forecast = invert_scale(scaler, forecast)
        # invert differencing
        forecast = invert_difference(prev[i], forecast, diff)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [19]:
# make one forecast with an LSTM
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # convert to array
    return [x for x in forecast[0, :]][0]
# evaluate the model
def make_lstm(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff):
    forecasts = []
    prev = raw_value[-(test_size+diff):(len(raw_value)-diff)]  # Y_(t-diff)
    for i in range(len(test)):
        X = test[i, 0:n_lag]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # invert scaling
        forecast = invert_scale(scaler, forecast)
        # invert differencing
        forecast = invert_difference(prev[i], forecast)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [20]:
# make one forecast with a hybird model
def forecast_comb(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    Z = X.reshape(1, 1, len(X))
    X = X.reshape(1, len(X))
    # make forecast
    forecast = model.predict([X,Z], batch_size=n_batch)
    return [x for x in forecast[0, :]][0]
    return [x for x in forecast[0, :]][0]
# make forecasts in a row
def make_comb(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff):
    forecasts = []
    prev = raw_value[-(test_size+diff):(len(raw_value)-diff)]  # Y_(t-diff)
    for i in range(len(test)):
        X = test[i, 0:n_lag]
        # make forecast
        forecast = forecast_comb(model, X, n_batch)
        # invert scaling
        forecast = invert_scale(scaler, forecast)
        # invert differencing
        forecast = invert_difference(prev[i], forecast)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

## Evaluation with RMSE, MAE, MAPE

In [21]:
# evaluate RMSE for each step in forcasting
def evaluate_forecasts(truth, forecasts):
    truth = np.array(truth)
    forecasts = np.array(forecasts)
    mape = mean_absolute_percentage_error(truth, forecasts)*100
    return mape

# Perform on All Counties, LAST Trials

In [22]:
# given historical observations (t, t-1, t-2, … t-n+1) forecast t+1(future 1 day)
n_lag = 7  # number of lag, use 7 past days
n_seq = 1  # predict 1 future days
diff = 1 # order of difference
train_size = 63  # train set size
test_size = 18 # test set size

n_step = 7  # step size

n_batch = 1
nb_epoch = 100
n_neurons = 1

In [23]:
# for each date t
    # use [t, t + train_size) as training set
    # use [t + train_size, t + train_size + test_size) as testing set
    # evaluate by calculating RMSE

# holder for metrics:
MAPE = []
trainingTime = []
for county in counties:
    # cut a data trial from original dataset
    # prepare data
    s = extract(df1, county)
    raw_value = s[-(train_size+test_size+n_lag):]
    
    mape_AR, mape_LSTM, mape_double, mape_comb = [],[],[],[]
    for i in range(100):
        # for AR
        scaler, train, test = prepare_AR(raw_value, diff, train_size, test_size, n_lag, n_seq)
        # fit model
        model = ARIMA(train, order = (n_lag,0,0), trend = 'c')  # AR, I, MA
        model = model.fit()
        # make forecasts
        forecasts = make_AR(model, raw_value, test, test_size, n_lag, n_seq, diff)
        mape_AR.append(evaluate_forecasts(raw_value[-test_size:], forecasts))

        # for lstm
        scaler, train, test = prepare_nn(raw_value, diff, train_size, test_size, n_lag, n_seq)
        model = fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
        forecasts = make_lstm(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
        mape_LSTM.append(evaluate_forecasts(raw_value[-test_size:], forecasts))

        # for lstm(double)
        model = fit_double_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
        forecasts = make_lstm(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
        mape_double.append(evaluate_forecasts(raw_value[-test_size:], forecasts))

        # for hybrid
        model = fit_comb(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons)
        forecasts = make_comb(model, n_batch, raw_value, test, test_size, n_lag, n_seq, diff)
        mape_comb.append(evaluate_forecasts(raw_value[-test_size:], forecasts))

    MAPE.append([mape_AR, mape_LSTM, mape_double, mape_comb])

# AR

In [24]:
meanCI_AR = []
for i in range(len(counties)):
    MAPE_AR = MAPE[i][0]
    data = pd.DataFrame(MAPE_AR) # convert to df
    mean, h = mean_confidence_interval(data, confidence=0.95)
    meanCI_AR.append([mean,h])

In [25]:
meanCI_AR

[[4.221229646394697, array([0.])],
 [3.7486797823741616, array([8.85608507e-17])],
 [5.251342370628335, array([1.77121701e-16])],
 [4.7308951315109145, array([1.77121701e-16])],
 [3.475220333027791, array([8.85608507e-17])],
 [4.684604915054328, array([0.])],
 [4.143153191613755, array([0.])],
 [4.752056901095261, array([0.])]]

# LSTM

In [26]:
meanCI_LSTM = []
for i in range(len(counties)):
    MAPE_LSTM = MAPE[i][1]
    data = pd.DataFrame(MAPE_LSTM) # convert to df
    mean, h = mean_confidence_interval(data, confidence=0.95)
    meanCI_LSTM.append([mean,h])

In [27]:
meanCI_LSTM

[[3.7535974100997906, array([0.11244086])],
 [3.476215496707176, array([0.03499905])],
 [4.097200115541444, array([0.06747665])],
 [4.143279587506932, array([0.03025634])],
 [3.113757295858047, array([0.07560169])],
 [3.8308771355591618, array([0.05340304])],
 [3.563424987959894, array([0.02805286])],
 [3.418860377174695, array([0.08812663])]]

# LSTM(Double)

In [28]:
meanCI_double = []
for i in range(len(counties)):
    MAPE_double = MAPE[i][2]
    data = pd.DataFrame(MAPE_double) # convert to df
    mean, h = mean_confidence_interval(data, confidence=0.95)
    meanCI_double.append([mean,h])

In [29]:
meanCI_double

[[3.5457058144269693, array([0.16038408])],
 [3.553856464944356, array([0.1259008])],
 [4.115137972068289, array([0.08372642])],
 [4.138608170039517, array([0.08861026])],
 [3.394014143782596, array([0.1489043])],
 [3.7425188829290477, array([0.07628036])],
 [3.4871614454114224, array([0.05700479])],
 [3.304148610034888, array([0.1473473])]]

# Hybrid

In [30]:
meanCI_Hybrid = []
for i in range(len(counties)):
    MAPE_Hybrid = MAPE[i][-1]
    data = pd.DataFrame(MAPE_Hybrid) # convert to df
    mean, h = mean_confidence_interval(data, confidence=0.95)
    meanCI_Hybrid.append([mean,h])

In [31]:
meanCI_Hybrid

[[3.3476047498079025, array([0.031997])],
 [3.3282991246648272, array([0.02031813])],
 [4.136394338038747, array([0.02677222])],
 [4.16487260938828, array([0.0122634])],
 [2.9416041219276563, array([0.00946189])],
 [3.8885353081712477, array([0.02303819])],
 [3.4177239198143896, array([0.0177399])],
 [3.2027545276257428, array([0.04111248])]]